# FEW SHOT - CARE (com e sem memória)

CARE significa:
- Give the (CONTEXT)
- Describe (ACTION)
- Clarify the (RESULT)
- Give the (EXAMPLE)

**Objetivo:** Criar um sistema completo (de ponta a ponta) apenas utilizando os prompts

In [1]:
# Imports necessários
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.graph import START, MessagesState, StateGraph
from langchain_openai import ChatOpenAI

from IPython.display import display, Markdown
from pathlib import Path
import os
from dotenv import load_dotenv

In [2]:
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [4]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [5]:
# servirá para simular algumas respostas
llm_sem_memoria = ChatOpenAI(model="gpt-4o-mini")

## Aqui vou usar o template CARE

In [6]:
prompt = """
**Contexto:**
Você é um Product Manager  muito experiente que trabalha em uma fábrica de software. Sua tarefa é especificar um plano para a construção de um sistema (web e app) que sirva como um cardápio virtual. Esse sistema deve servir tanto aos restaurantes que o utilizam para o cadastro daquilo que servem (lanches, entradas, pratos principais, bebidas etc), quanto para os clientes dos restaurantes. Além disso, para os restaurantes deve existir um módulo para um dashboard para analisar os produtos.

**Ação:**
Crie um plando para construção desse sistema, com as etapas macro, equipes envolvidas, etapas que podem ser executadas em paralelo e quais aquelas que possuem predecessoras

**Resultado:**
Espera-se uma tabela (ou outro formato mais interessante) que represente este plano

**Exemplo (do que deve existir na saída):**
Etapa 1: [Nome da etapa] [Equipes] [Etapas Predecessoras] [Etapa que pode ser executada em paralelo]
Etapa 2: [Nome da etapa] [Equipes] [Etapas Predecessoras] [Etapa que pode ser executada em paralelo]
...
Etapa n: [Nome da etapa] [Equipes] [Etapas Predecessoras] [Etapa que pode ser executada em paralelo]"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


**Contexto:**
Você é um Product Manager  muito experiente que trabalha em uma fábrica de software. Sua tarefa é especificar um plano para a construção de um sistema (web e app) que sirva como um cardápio virtual. Esse sistema deve servir tanto aos restaurantes que o utilizam para o cadastro daquilo que servem (lanches, entradas, pratos principais, bebidas etc), quanto para os clientes dos restaurantes. Além disso, para os restaurantes deve existir um módulo para um dashboard para analisar os produtos.

**Ação:**
Crie um plando para construção desse sistema, com as etapas macro, equipes envolvidas, etapas que podem ser executadas em paralelo e quais aquelas que possuem predecessoras

**Resultado:**
Espera-se uma tabela (ou outro formato mais interessante) que represente este plano

**Exemplo (do que deve existir na saída):**
Etapa 1: [Nome da etapa] [Equipes] [Etapas Predecessoras] [Etapa que pode ser executada em paralelo]
Etapa 2: [Nome da etapa] [Equipes] [Etapas Predecessoras] [Etapa que pode ser executada em paralelo]
...
Etapa n: [Nome da etapa] [Equipes] [Etapas Predecessoras] [Etapa que pode ser executada em paralelo]

----

*Resposta:*

----

Abaixo está um plano estruturado para a construção de um sistema de cardápio virtual, organizado em formato de tabela. As etapas incluem as equipes envolvidas, as etapas predecessoras que precisam ser concluídas antes do início de cada etapa e as etapas que podem ser executadas em paralelo:

| **Etapa** | **Nome da Etapa**                       | **Equipes**                          | **Etapas Predecessoras**      | **Etapas que podem ser executadas em paralelo** |
|-----------|-----------------------------------------|-------------------------------------|-------------------------------|------------------------------------------------|
| 1         | Levantamento de Requisitos              | Product Management, UX/UI Design    | Nenhuma                       | Equipe de Desenvolvimento (Definição de Arquitetura) |
| 2         | Definição da Arquitetura                | Desenvolvimento, Product Management  | Etapa 1                       | Equipe de Design (Wireframes e Prototipagem)  |
| 3         | Wireframes e Prototipagem               | UX/UI Design                        | Etapa 1                       | Equipe de Desenvolvimento (Definição de Arquitetura) |
| 4         | Desenvolvimento da Interface do Usuário  | Desenvolvimento, UX/UI Design       | Etapas 2, 3                   | Nenhuma                                          |
| 5         | Implementação da Lógica de Negócio      | Desenvolvimento                     | Etapa 2                       | Equipe de QA (Teste de Integração)              |
| 6         | Desenvolvimento do Módulo de Dashboard   | Desenvolvimento                     | Etapa 2                       | Equipe de QA (Teste de Integração)              |
| 7         | Integração do Sistema                    | Desenvolvimento                     | Etapas 4, 5, 6                | Equipe de QA (Teste de Integração)              |
| 8         | Testes de Funcionalidade e Usabilidade   | QA, UX/UI Design                    | Etapa 7                       | Nenhuma                                          |
| 9         | Implementação de Feedback dos Testes     | Desenvolvimento, QA                 | Etapa 8                       | Nenhuma                                          |
| 10        | Preparação para Lançamento               | Product Management, Marketing       | Etapas 8, 9                   | Nenhuma                                          |
| 11        | Lançamento do Sistema                    | Todos os times                      | Etapa 10                      | Nenhuma                                          |
| 12        | Monitoramento Pós-Lançamento             | Suporte, Product Management         | Etapa 11                      | Nenhuma                                          |

### Descrição das Etapas

1. **Levantamento de Requisitos**: Coletar informações sobre necessidades dos restaurantes e clientes.
2. **Definição da Arquitetura**: Estabelecer a estrutura técnica do sistema, incluindo tecnologias a serem utilizadas.
3. **Wireframes e Prototipagem**: Criar maquetes de tela para a interface do usuário e protótipos interativos.
4. **Desenvolvimento da Interface do Usuário**: Codificação das interfaces de usuário para aplicativos e web.
5. **Implementação da Lógica de Negócio**: Desenvolvimento das regras que operacionalizam o sistema.
6. **Desenvolvimento do Módulo de Dashboard**: Implementação de funcionalidades de análise e relatórios para restaurantes.
7. **Integração do Sistema**: Conexão entre todos os componentes do sistema para funcionalidade completa.
8. **Testes de Funcionalidade e Usabilidade**: Validar o sistema para detectar bugs e melhorar a experiência do usuário.
9. **Implementação de Feedback dos Testes**: Fazer ajustes conforme as descobertas dos testes realizados.
10. **Preparação para Lançamento**: Planejar e executar ações de marketing e pré-lançamento.
11. **Lançamento do Sistema**: Disponibilizar o sistema para o público-alvo.
12. **Monitoramento Pós-Lançamento**: Monitorar a performance do sistema e coletar feedback dos usuários.

Esse plano proporciona uma visão clara das etapas necessárias para o desenvolvimento do sistema de cardápio virtual, destacando a interdependência entre as equipes e as atividades.

----

## Vamos transformar a resposta anterior

In [7]:
prompt = """
**Transforme:**
Transforme este plano em um esquema de cronograma, onde as colunas representam 
mês 1, mês 2, e assim sucessivamente, e as linhas são as etapas
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


**Transforme:**
Transforme este plano em um esquema de cronograma, onde as colunas representam 
mês 1, mês 2, e assim sucessivamente, e as linhas são as etapas


----

*Resposta:*

----

Aqui está um esquema de cronograma organizado por meses, onde as colunas representam os meses de desenvolvimento e as linhas representam as etapas do plano. O cronograma assume uma duração aproximada para cada etapa. Note que o número de meses e a duração de cada etapa podem ser ajustados de acordo com a programação real do projeto.

| **Etapas**                                | **Mês 1**  | **Mês 2**  | **Mês 3**  | **Mês 4**  | **Mês 5**  | **Mês 6**  |
|-------------------------------------------|------------|------------|------------|------------|------------|------------|
| 1. Levantamento de Requisitos             | ███████    |            |            |            |            |            |
| 2. Definição da Arquitetura               |            | ██████     |            |            |            |            |
| 3. Wireframes e Prototipagem              |            | ██████     |            |            |            |            |
| 4. Desenvolvimento da Interface do Usuário |            |            | █████████  |            |            |            |
| 5. Implementação da Lógica de Negócio     |            |            | ██         | ████       |            |            |
| 6. Desenvolvimento do Módulo de Dashboard  |            |            | ██         | ████       |            |            |
| 7. Integração do Sistema                   |            |            |            | ██████     |            |            |
| 8. Testes de Funcionalidade e Usabilidade  |            |            |            |            | █████████  |            |
| 9. Implementação de Feedback dos Testes    |            |            |            |            | ██         | ████       |
| 10. Preparação para Lançamento            |            |            |            |            |            | ██████     |
| 11. Lançamento do Sistema                  |            |            |            |            |            | ████       |
| 12. Monitoramento Pós-Lançamento           |            |            |            |            |            | ██         |

### Explicações

- As células preenchidas (█) representam o período em que uma determinada etapa está em andamento.
- Levantamento de Requisitos é feito no primeiro mês.
- Definição da Arquitetura e Wireframes e Prototipagem são realizadas simultaneamente no segundo mês.
- O desenvolvimento da interface do usuário ocorre no terceiro mês.
- A implementação da lógica de negócio e o desenvolvimento do módulo de dashboard acontecem em paralelo no final do terceiro e início do quarto mês.
- A integração do sistema é realizada no quarto mês, seguida pelos testes de funcionalidade e usabilidade no quinto mês.
- A implementação do feedback dos testes é feita no final do quinto mês até o início do sexto.
- A preparação para o lançamento é realizada no sexto mês, seguida pelo lançamento do sistema.
- O monitoramento pós-lançamento ocorre após o lançamento, no final do sexto mês.

Observação: Este cronograma é uma representação simplificada e pode variar dependendo de diversos fatores, como a complexidade do sistema, disponibilidade de recursos e outras prioridades do projeto.

----

## Etapa 1 - Levantamento de Requisitos

In [8]:
prompt = """
**Contexto:**
Haja como a equipe de desenvolvimento (juntamente com o Product Management e UX/UI Design) 
que está acompanhando os representantes dos restaurantes para a execução da etapa 1 - Levantamento de Requisitos

Ação:
Crie uma lista de perguntas que devem ser feitas da equipe de desenvolvimento para a 
equipe de representantes dos restaurantes

Resultado:
Essa lista ajudará a guiar o levamentamento de requisitos para a construção o 
sistema de cadápio. Seja o mais abrangente possível

Exemplo (do formato da lista de perguntas):
Pergunta 1: [Escopo da pergunta] [Pergunta]
Pergunta 2: [Escopo da pergunta] [Pergunta]
....
Pergunta n: [Escopo da pergunta] [Pergunta]"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


**Contexto:**
Haja como a equipe de desenvolvimento (juntamente com o Product Management e UX/UI Design) 
que está acompanhando os representantes dos restaurantes para a execução da etapa 1 - Levantamento de Requisitos

Ação:
Crie uma lista de perguntas que devem ser feitas da equipe de desenvolvimento para a 
equipe de representantes dos restaurantes

Resultado:
Essa lista ajudará a guiar o levamentamento de requisitos para a construção o 
sistema de cadápio. Seja o mais abrangente possível

Exemplo (do formato da lista de perguntas):
Pergunta 1: [Escopo da pergunta] [Pergunta]
Pergunta 2: [Escopo da pergunta] [Pergunta]
....
Pergunta n: [Escopo da pergunta] [Pergunta]

----

*Resposta:*

----

Aqui está uma lista abrangente de perguntas que a equipe de desenvolvimento, juntamente com o Product Management e o UX/UI Design, pode fazer aos representantes dos restaurantes durante a etapa de Levantamento de Requisitos para a construção do sistema de cardápio virtual:

### Lista de Perguntas

**Pergunta 1: Necessidades Gerais**  
Quais são seus principais objetivos ao utilizar um sistema de cardápio virtual?

**Pergunta 2: Público-Alvo**  
Quem são os principais usuários do sistema? (Ex: clientes, funcionários, gerentes)

**Pergunta 3: Funcionalidade do Cardápio**  
Quais tipos de itens você gostaria de incluir no cardápio? (Ex: lanches, bebidas, sobremesas, promoções)

**Pergunta 4: Cadastro de Itens**  
Como você gostaria de gerenciar o cadastro de novos itens no cardápio? (Ex: manualmente, importação de dados)

**Pergunta 5: Descrições dos Produtos**  
Que informações você gostaria de incluir para cada item no cardápio? (Ex: descrição, preço, ingredientes, alérgenos)

**Pergunta 6: Imagens dos Produtos**  
As fotos dos itens são importantes para você? Se sim, qual a quantidade de imagens e a qualidade desejada?

**Pergunta 7: Atualizações em Tempo Real**  
Com que frequência você precisa atualizar o cardápio? (Ex: diariamente, semanalmente)

**Pergunta 8: Promoções e Destaques**  
Como você gostaria de apresentar promoções ou itens em destaque no cardápio?

**Pergunta 9: Opções de Personalização**  
Os clientes podem personalizar pedidos? Se sim, quais opções são importantes? (Ex: escolher ingredientes)

**Pergunta 10: Acessibilidade**  
O cardápio precisa estar disponível em diferentes idiomas? Quais idiomas são necessários?

**Pergunta 11: Dashboard de Análises**  
Que métricas ou dados você gostaria de visualizar no dashboard? (Ex: vendas por item, popularidade de pratos)

**Pergunta 12: Relatórios**  
Você precisa de relatórios periódicos? Quais informações eles devem conter?

**Pergunta 13: Integrações**  
Você utiliza outras ferramentas ou sistemas (ex: contabilidade, estoque) que precisarão ser integrados ao novo sistema?

**Pergunta 14: Autenticação e Segurança**  
Quais são suas necessidades em relação ao acesso do sistema? (Ex: múltiplos níveis de usuário, autenticação de dois fatores)

**Pergunta 15: Suporte a Múltiplas Localizações**  
Se o seu restaurante tiver várias localizações, como você gostaria de gerenciar o cardápio para cada uma?

**Pergunta 16: Feedback dos Clientes**  
Você gostaria de incluir uma funcionalidade que permita aos clientes fornecer feedback sobre os itens do cardápio?

**Pergunta 17: Experiência do Usuário**  
Quais características de design e experiência de usuário você considera essenciais para um cardápio virtual?

**Pergunta 18: Configurações e Preferências do Usuário**  
Como você gostaria que os usuários configurassem suas preferências e opções de visualização do cardápio?

**Pergunta 19: Operação Offline**  
Haverá necessidade de acesso offline ao sistema? Se sim, em qual situação isso seria relevante?

**Pergunta 20: Tempo e Orçamento**  
Qual é o seu prazo ideal para ter o sistema implementado e operando? Existe um orçamento específico que devemos considerar?

Essas perguntas devem ajudar a coletar informações valiosas sobre as necessidades e expectativas dos restaurantes em relação ao sistema de cardápio virtual, garantindo que o produto final atenda suas demandas.

----

### Simulando a resposta dos representantes dos restaurantes (LLM sem memória)

In [10]:
prompt = """
Haja com um membro de Representantes de Restaurantes que estejam interessados na construção
de um sistema de cardápio virtual. Você deverá responder o e-mail abaixo de forma a sanar as dúvidas
da equipe de desenvolvimento:

-------------

Olá, bom dia!
Somos da equipe de desenvolvimento da fábrica de software ACME.
Estamos desenvolvendo o sistema de cardápio virtual e gostaríamos 
de tirar algumas dúvidas a respeito do sistema.

Seguem as dúvidas,
Atenciosamente,
Equipe de SW ACME


Lista de Perguntas:
Pergunta 1: Necessidades Gerais
Quais são seus principais objetivos ao utilizar um sistema de cardápio virtual?

Pergunta 2: Público-Alvo
Quem são os principais usuários do sistema? (Ex: clientes, funcionários, gerentes)

Pergunta 3: Funcionalidade do Cardápio
Quais tipos de itens você gostaria de incluir no cardápio? (Ex: lanches, bebidas, sobremesas, promoções)

Pergunta 4: Cadastro de Itens
Como você gostaria de gerenciar o cadastro de novos itens no cardápio? (Ex: manualmente, importação de dados)

Pergunta 5: Descrições dos Produtos
Que informações você gostaria de incluir para cada item no cardápio? (Ex: descrição, preço, ingredientes, alérgenos)

Pergunta 6: Imagens dos Produtos
As fotos dos itens são importantes para você? Se sim, qual a quantidade de imagens e a qualidade desejada?

Pergunta 7: Atualizações em Tempo Real
Com que frequência você precisa atualizar o cardápio? (Ex: diariamente, semanalmente)

Pergunta 8: Promoções e Destaques
Como você gostaria de apresentar promoções ou itens em destaque no cardápio?

Pergunta 9: Opções de Personalização
Os clientes podem personalizar pedidos? Se sim, quais opções são importantes? (Ex: escolher ingredientes)

Pergunta 10: Acessibilidade
O cardápio precisa estar disponível em diferentes idiomas? Quais idiomas são necessários?

Pergunta 11: Dashboard de Análises
Que métricas ou dados você gostaria de visualizar no dashboard? (Ex: vendas por item, popularidade de pratos)

Pergunta 12: Relatórios
Você precisa de relatórios periódicos? Quais informações eles devem conter?

Pergunta 13: Integrações
Você utiliza outras ferramentas ou sistemas (ex: contabilidade, estoque) que precisarão ser integrados ao novo sistema?

Pergunta 14: Autenticação e Segurança
Quais são suas necessidades em relação ao acesso do sistema? (Ex: múltiplos níveis de usuário, autenticação de dois fatores)

Pergunta 15: Suporte a Múltiplas Localizações
Se o seu restaurante tiver várias localizações, como você gostaria de gerenciar o cardápio para cada uma?

Pergunta 16: Feedback dos Clientes
Você gostaria de incluir uma funcionalidade que permita aos clientes fornecer feedback sobre os itens do cardápio?

Pergunta 17: Experiência do Usuário
Quais características de design e experiência de usuário você considera essenciais para um cardápio virtual?

Pergunta 18: Configurações e Preferências do Usuário
Como você gostaria que os usuários configurassem suas preferências e opções de visualização do cardápio?

Pergunta 19: Operação Offline
Haverá necessidade de acesso offline ao sistema? Se sim, em qual situação isso seria relevante?

Pergunta 20: Tempo e Orçamento
Qual é o seu prazo ideal para ter o sistema implementado e operando? Existe um orçamento específico que devemos considerar?

"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

response = llm_sem_memoria.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Prompt:*

----


Haja com um membro de Representantes de Restaurantes que estejam interessados na construção
de um sistema de cardápio virtual. Você deverá responder o e-mail abaixo de forma a sanar as dúvidas
da equipe de desenvolvimento:

-------------

Olá, bom dia!
Somos da equipe de desenvolvimento da fábrica de software ACME.
Estamos desenvolvendo o sistema de cardápio virtual e gostaríamos 
de tirar algumas dúvidas a respeito do sistema.

Seguem as dúvidas,
Atenciosamente,
Equipe de SW ACME


Lista de Perguntas:
Pergunta 1: Necessidades Gerais
Quais são seus principais objetivos ao utilizar um sistema de cardápio virtual?

Pergunta 2: Público-Alvo
Quem são os principais usuários do sistema? (Ex: clientes, funcionários, gerentes)

Pergunta 3: Funcionalidade do Cardápio
Quais tipos de itens você gostaria de incluir no cardápio? (Ex: lanches, bebidas, sobremesas, promoções)

Pergunta 4: Cadastro de Itens
Como você gostaria de gerenciar o cadastro de novos itens no cardápio? (Ex: manualmente, importação de dados)

Pergunta 5: Descrições dos Produtos
Que informações você gostaria de incluir para cada item no cardápio? (Ex: descrição, preço, ingredientes, alérgenos)

Pergunta 6: Imagens dos Produtos
As fotos dos itens são importantes para você? Se sim, qual a quantidade de imagens e a qualidade desejada?

Pergunta 7: Atualizações em Tempo Real
Com que frequência você precisa atualizar o cardápio? (Ex: diariamente, semanalmente)

Pergunta 8: Promoções e Destaques
Como você gostaria de apresentar promoções ou itens em destaque no cardápio?

Pergunta 9: Opções de Personalização
Os clientes podem personalizar pedidos? Se sim, quais opções são importantes? (Ex: escolher ingredientes)

Pergunta 10: Acessibilidade
O cardápio precisa estar disponível em diferentes idiomas? Quais idiomas são necessários?

Pergunta 11: Dashboard de Análises
Que métricas ou dados você gostaria de visualizar no dashboard? (Ex: vendas por item, popularidade de pratos)

Pergunta 12: Relatórios
Você precisa de relatórios periódicos? Quais informações eles devem conter?

Pergunta 13: Integrações
Você utiliza outras ferramentas ou sistemas (ex: contabilidade, estoque) que precisarão ser integrados ao novo sistema?

Pergunta 14: Autenticação e Segurança
Quais são suas necessidades em relação ao acesso do sistema? (Ex: múltiplos níveis de usuário, autenticação de dois fatores)

Pergunta 15: Suporte a Múltiplas Localizações
Se o seu restaurante tiver várias localizações, como você gostaria de gerenciar o cardápio para cada uma?

Pergunta 16: Feedback dos Clientes
Você gostaria de incluir uma funcionalidade que permita aos clientes fornecer feedback sobre os itens do cardápio?

Pergunta 17: Experiência do Usuário
Quais características de design e experiência de usuário você considera essenciais para um cardápio virtual?

Pergunta 18: Configurações e Preferências do Usuário
Como você gostaria que os usuários configurassem suas preferências e opções de visualização do cardápio?

Pergunta 19: Operação Offline
Haverá necessidade de acesso offline ao sistema? Se sim, em qual situação isso seria relevante?

Pergunta 20: Tempo e Orçamento
Qual é o seu prazo ideal para ter o sistema implementado e operando? Existe um orçamento específico que devemos considerar?



----

*Resposta:*

----

Olá, equipe da ACME,

Bom dia! Agradeço por entrar em contato e pela oportunidade de esclarecer as dúvidas sobre o sistema de cardápio virtual. Vamos às perguntas:

**1. Necessidades Gerais**  
Nossos principais objetivos ao utilizar um sistema de cardápio virtual incluem oferecer uma experiência melhor aos clientes, aumentar a agilidade no processo de pedidos e facilitar a atualização e gerenciamento dos itens do cardápio.

**2. Público-Alvo**  
Os principais usuários do sistema são clientes, funcionários do restaurante e gerentes. O foco está em proporcionar uma experiência intuitiva para os clientes, ao mesmo tempo em que facilita o trabalho dos funcionários na cozinha e atendimento.

**3. Funcionalidade do Cardápio**  
Gostaríamos de incluir uma variedade de itens, como lanches, bebidas, sobremesas e promoções. A capacidade de destacar pratos específicos e opções sazonais seria um adicional benéfico.

**4. Cadastro de Itens**  
Preferimos gerenciar o cadastro de novos itens manualmente, mas também gostaríamos de ter uma opção para importar dados quando necessário, especialmente ao adicionar novos produtos em massa.

**5. Descrições dos Produtos**  
Para cada item no cardápio, gostaríamos de incluir descrição, preço, ingredientes e informações sobre alérgenos. Isso ajuda os clientes a fazer escolhas informadas.

**6. Imagens dos Produtos**  
As fotos dos itens são extremamente importantes. Idealmente, gostaríamos de incluir de 1 a 3 imagens de cada item, com qualidade alta para destacar a apresentação dos pratos.

**7. Atualizações em Tempo Real**  
Precisamos atualizar o cardápio com frequência semanal, ou conforme necessário, por conta de alterações de estoque e novas promoções.

**8. Promoções e Destaques**  
As promoções ou itens em destaque devem ser apresentados de forma clara, talvez com uma seção específica no topo do cardápio ou com um destaque visual que chame a atenção dos clientes.

**9. Opções de Personalização**  
Sim, os clientes devem poder personalizar pedidos. É essencial ter opções para escolher ingredientes e adaptações em pratos, como substituições ou remoções de itens.

**10. Acessibilidade**  
O cardápio deve estar disponível em português e inglês inicialmente. Se o público internacional crescer, outras línguas poderão ser consideradas.

**11. Dashboard de Análises**  
Gostaríamos de visualizar métricas como vendas por item, popularidade de pratos e dados demográficos dos clientes. Essas informações são cruciais para o planejamento de estoque e promoções.

**12. Relatórios**  
Sim, precisamos de relatórios periódicos que incluam vendas totais, itens mais vendidos e retorno sobre promoções, com foco em informações que ajudem a tomar decisões estratégicas.

**13. Integrações**  
Atualmente usamos um sistema de contabilidade que precisará ser integrado ao novo sistema para facilitar o gerenciamento financeiro e de estoque.

**14. Autenticação e Segurança**  
É importante ter múltiplos níveis de usuário, permitindo diferentes permissões de acesso. A autenticação de dois fatores também é desejável para segurança adicional.

**15. Suporte a Múltiplas Localizações**  
Se tivermos várias localizações, gostaríamos de gerenciar o cardápio centralmente, mas com a flexibilidade de personalizar itens para diferentes locais conforme a necessidade.

**16. Feedback dos Clientes**  
Sim, gostaríamos de incluir uma funcionalidade para que os clientes possam fornecer feedback sobre os itens do cardápio. Isso nos ajudará a melhorar continuamente nossos produtos.

**17. Experiência do Usuário**  
Características essenciais incluem um design limpo e atrativo, navegação intuitiva e carregamento rápido. O sistema deve ser responsivo e bem otimizado para dispositivos móveis.

**18. Configurações e Preferências do Usuário**  
Desejamos que os usuários possam configurar suas preferências de visualização, como categoria de itens ou filtros para opções vegetarianas, por exemplo.

**19. Operação Offline**  
Sim, o acesso offline será relevante, especialmente em situações de conexão instável. Isso permitiria que a equipe continuasse a operar em momentos críticos.

**20. Tempo e Orçamento**  
Idealmente, gostaríamos de ter o sistema implementado em um prazo de três meses. O orçamento a ser considerado está em discussão, mas buscamos uma solução que atenda nossas necessidades sem comprometer a qualidade.

Estou à disposição para quaisquer outras dúvidas ou esclarecimentos que precisem. 

Atenciosamente,  
[Seu Nome]  
[Seu Cargo]  
[Nome do Restaurante]  
[Seu Contato] 

----

## Etapa 2 - Definição da Arquitetura

In [11]:
prompt = """
**Contexto:**
Haja como a equipe de desenvolvimento (juntamente com o Product Management) 
que está acompanhando os representantes dos restaurantes para a execução da etapa 2 - Definição da Arquitetura

Ação:
Faça uma definição da arquitetura do sistema, de forma a atender os principais requisitos que serão listados no final.

Resultado:
A definição da arquitetura ajudará ao arquiteto de software a tomar as melhores decisões.

Exemplo (do formato da lista de perguntas):
Definição 1: [Componente] [Explicação do componente]
Definição 2: [Componente] [Explicação do componente]
Definição n: [Componente] [Explicação do componente]

Segue os principais requisitos informados pelos representantes dos restaurantes:

1. Necessidades Gerais
Nossos principais objetivos ao utilizar um sistema de cardápio virtual incluem oferecer uma experiência melhor aos clientes, aumentar a agilidade no processo de pedidos e facilitar a atualização e gerenciamento dos itens do cardápio.

2. Público-Alvo
Os principais usuários do sistema são clientes, funcionários do restaurante e gerentes. O foco está em proporcionar uma experiência intuitiva para os clientes, ao mesmo tempo em que facilita o trabalho dos funcionários na cozinha e atendimento.

3. Funcionalidade do Cardápio
Gostaríamos de incluir uma variedade de itens, como lanches, bebidas, sobremesas e promoções. A capacidade de destacar pratos específicos e opções sazonais seria um adicional benéfico.

4. Cadastro de Itens
Preferimos gerenciar o cadastro de novos itens manualmente, mas também gostaríamos de ter uma opção para importar dados quando necessário, especialmente ao adicionar novos produtos em massa.

5. Descrições dos Produtos
Para cada item no cardápio, gostaríamos de incluir descrição, preço, ingredientes e informações sobre alérgenos. Isso ajuda os clientes a fazer escolhas informadas.

6. Imagens dos Produtos
As fotos dos itens são extremamente importantes. Idealmente, gostaríamos de incluir de 1 a 3 imagens de cada item, com qualidade alta para destacar a apresentação dos pratos.

7. Atualizações em Tempo Real
Precisamos atualizar o cardápio com frequência semanal, ou conforme necessário, por conta de alterações de estoque e novas promoções.

8. Promoções e Destaques
As promoções ou itens em destaque devem ser apresentados de forma clara, talvez com uma seção específica no topo do cardápio ou com um destaque visual que chame a atenção dos clientes.

9. Opções de Personalização
Sim, os clientes devem poder personalizar pedidos. É essencial ter opções para escolher ingredientes e adaptações em pratos, como substituições ou remoções de itens.

10. Acessibilidade
O cardápio deve estar disponível em português e inglês inicialmente. Se o público internacional crescer, outras línguas poderão ser consideradas.

11. Dashboard de Análises
Gostaríamos de visualizar métricas como vendas por item, popularidade de pratos e dados demográficos dos clientes. Essas informações são cruciais para o planejamento de estoque e promoções.

12. Relatórios
Sim, precisamos de relatórios periódicos que incluam vendas totais, itens mais vendidos e retorno sobre promoções, com foco em informações que ajudem a tomar decisões estratégicas.

13. Integrações
Atualmente usamos um sistema de contabilidade que precisará ser integrado ao novo sistema para facilitar o gerenciamento financeiro e de estoque.

14. Autenticação e Segurança
É importante ter múltiplos níveis de usuário, permitindo diferentes permissões de acesso. A autenticação de dois fatores também é desejável para segurança adicional.

15. Suporte a Múltiplas Localizações
Se tivermos várias localizações, gostaríamos de gerenciar o cardápio centralmente, mas com a flexibilidade de personalizar itens para diferentes locais conforme a necessidade.

16. Feedback dos Clientes
Sim, gostaríamos de incluir uma funcionalidade para que os clientes possam fornecer feedback sobre os itens do cardápio. Isso nos ajudará a melhorar continuamente nossos produtos.

17. Experiência do Usuário
Características essenciais incluem um design limpo e atrativo, navegação intuitiva e carregamento rápido. O sistema deve ser responsivo e bem otimizado para dispositivos móveis.

18. Configurações e Preferências do Usuário
Desejamos que os usuários possam configurar suas preferências de visualização, como categoria de itens ou filtros para opções vegetarianas, por exemplo.

19. Operação Offline
Sim, o acesso offline será relevante, especialmente em situações de conexão instável. Isso permitiria que a equipe continuasse a operar em momentos críticos.

20. Tempo e Orçamento
Idealmente, gostaríamos de ter o sistema implementado em um prazo de três meses. O orçamento a ser considerado está em discussão, mas buscamos uma solução que atenda nossas necessidades sem comprometer a qualidade.

"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


**Contexto:**
Haja como a equipe de desenvolvimento (juntamente com o Product Management) 
que está acompanhando os representantes dos restaurantes para a execução da etapa 2 - Definição da Arquitetura

Ação:
Faça uma definição da arquitetura do sistema, de forma a atender os principais requisitos que serão listados no final.

Resultado:
A definição da arquitetura ajudará ao arquiteto de software a tomar as melhores decisões.

Exemplo (do formato da lista de perguntas):
Definição 1: [Componente] [Explicação do componente]
Definição 2: [Componente] [Explicação do componente]
Definição n: [Componente] [Explicação do componente]

Segue os principais requisitos informados pelos representantes dos restaurantes:

1. Necessidades Gerais
Nossos principais objetivos ao utilizar um sistema de cardápio virtual incluem oferecer uma experiência melhor aos clientes, aumentar a agilidade no processo de pedidos e facilitar a atualização e gerenciamento dos itens do cardápio.

2. Público-Alvo
Os principais usuários do sistema são clientes, funcionários do restaurante e gerentes. O foco está em proporcionar uma experiência intuitiva para os clientes, ao mesmo tempo em que facilita o trabalho dos funcionários na cozinha e atendimento.

3. Funcionalidade do Cardápio
Gostaríamos de incluir uma variedade de itens, como lanches, bebidas, sobremesas e promoções. A capacidade de destacar pratos específicos e opções sazonais seria um adicional benéfico.

4. Cadastro de Itens
Preferimos gerenciar o cadastro de novos itens manualmente, mas também gostaríamos de ter uma opção para importar dados quando necessário, especialmente ao adicionar novos produtos em massa.

5. Descrições dos Produtos
Para cada item no cardápio, gostaríamos de incluir descrição, preço, ingredientes e informações sobre alérgenos. Isso ajuda os clientes a fazer escolhas informadas.

6. Imagens dos Produtos
As fotos dos itens são extremamente importantes. Idealmente, gostaríamos de incluir de 1 a 3 imagens de cada item, com qualidade alta para destacar a apresentação dos pratos.

7. Atualizações em Tempo Real
Precisamos atualizar o cardápio com frequência semanal, ou conforme necessário, por conta de alterações de estoque e novas promoções.

8. Promoções e Destaques
As promoções ou itens em destaque devem ser apresentados de forma clara, talvez com uma seção específica no topo do cardápio ou com um destaque visual que chame a atenção dos clientes.

9. Opções de Personalização
Sim, os clientes devem poder personalizar pedidos. É essencial ter opções para escolher ingredientes e adaptações em pratos, como substituições ou remoções de itens.

10. Acessibilidade
O cardápio deve estar disponível em português e inglês inicialmente. Se o público internacional crescer, outras línguas poderão ser consideradas.

11. Dashboard de Análises
Gostaríamos de visualizar métricas como vendas por item, popularidade de pratos e dados demográficos dos clientes. Essas informações são cruciais para o planejamento de estoque e promoções.

12. Relatórios
Sim, precisamos de relatórios periódicos que incluam vendas totais, itens mais vendidos e retorno sobre promoções, com foco em informações que ajudem a tomar decisões estratégicas.

13. Integrações
Atualmente usamos um sistema de contabilidade que precisará ser integrado ao novo sistema para facilitar o gerenciamento financeiro e de estoque.

14. Autenticação e Segurança
É importante ter múltiplos níveis de usuário, permitindo diferentes permissões de acesso. A autenticação de dois fatores também é desejável para segurança adicional.

15. Suporte a Múltiplas Localizações
Se tivermos várias localizações, gostaríamos de gerenciar o cardápio centralmente, mas com a flexibilidade de personalizar itens para diferentes locais conforme a necessidade.

16. Feedback dos Clientes
Sim, gostaríamos de incluir uma funcionalidade para que os clientes possam fornecer feedback sobre os itens do cardápio. Isso nos ajudará a melhorar continuamente nossos produtos.

17. Experiência do Usuário
Características essenciais incluem um design limpo e atrativo, navegação intuitiva e carregamento rápido. O sistema deve ser responsivo e bem otimizado para dispositivos móveis.

18. Configurações e Preferências do Usuário
Desejamos que os usuários possam configurar suas preferências de visualização, como categoria de itens ou filtros para opções vegetarianas, por exemplo.

19. Operação Offline
Sim, o acesso offline será relevante, especialmente em situações de conexão instável. Isso permitiria que a equipe continuasse a operar em momentos críticos.

20. Tempo e Orçamento
Idealmente, gostaríamos de ter o sistema implementado em um prazo de três meses. O orçamento a ser considerado está em discussão, mas buscamos uma solução que atenda nossas necessidades sem comprometer a qualidade.



----

*Resposta:*

----

Aqui está uma definição da arquitetura do sistema de cardápio virtual, estruturada em componentes. Cada definição inclui uma explicação para ajudar a entender como cada parte atende aos requisitos listados.

### Definição da Arquitetura do Sistema

**Definição 1: Frontend (Interface do Usuário)**  
**Componente:** Aplicativo Web e Móvel  
**Explicação do componente:** A interface do usuário será desenvolvida usando tecnologias modernas como React para a web e React Native para dispositivos móveis, garantindo uma experiência responsiva e fluida. Este componente deve proporcionar uma navegação intuitiva, fácil acesso às opções de cardápio e personalização de pedidos, alinhando-se aos requisitos de design e experiência do usuário.

---

**Definição 2: Backend (Lógica de Negócio)**  
**Componente:** API RESTful  
**Explicação do componente:** O backend será construído utilizando uma arquitetura RESTful em Node.js com Express, permitindo uma comunicação eficiente entre o frontend e o banco de dados. Esta arquitetura suportará operações CRUD (Criar, Ler, Atualizar, Deletar) para o gerenciamento do cardápio, promoções e feedback dos clientes, garantindo que as necessidades de atualização em tempo real e cadastro sejam atendidas.

---

**Definição 3: Banco de Dados**  
**Componente:** Banco de Dados Relacional (PostgreSQL)  
**Explicação do componente:** Um banco de dados relational será utilizado para armazenar informações sobre itens do cardápio, usuários, pedidos, feedback e relatórios. O PostgreSQL oferece robustez e escalabilidade, permitindo o gerenciamento centralizado do cardápio e suporte a múltiplas localizações, conforme necessário.

---

**Definição 4: Sistema de Autenticação**  
**Componente:** Autenticação JWT (JSON Web Token)  
**Explicação do componente:** Para gerenciar o acesso seguro ao sistema, será implementado um sistema de autenticação baseado em JWT, que inclui múltiplos níveis de usuário (clientes, funcionários, gerentes) e a opção de autenticação de dois fatores. Isso vai de encontro aos requisitos de segurança e proteção de dados sensíveis.

---

**Definição 5: Sistema de Upload de Imagens**  
**Componente:** Armazenamento em Nuvem (Amazon S3)  
**Explicação do componente:** Para armazenar e servir as imagens dos pratos em alta qualidade de forma eficiente, utilizaremos uma solução de armazenamento em nuvem, como o Amazon S3. Isso garante que as imagens possam ser carregadas rapidamente e sejam facilmente gerenciáveis, conforme necessário para cada item do cardápio.

---

**Definição 6: Integração com Sistemas Externos**  
**Componente:** API para Integração com Sistemas de Contabilidade  
**Explicação do componente:** Este componente permitirá a comunicação entre o novo sistema de cardápio e o sistema de contabilidade existente, utilizando APIs REST para troca de dados de vendas e estoque, garantindo eficiência no gerenciamento financeiro e operacional.

---

**Definição 7: Módulo de Análises e Relatórios**  
**Componente:** Ferramenta de BI (Business Intelligence)  
**Explicação do componente:** Um módulo de análises será integrado ao backend para coletar e apresentar métricas como vendas, popularidade de pratos e dados demográficos dos clientes. Usaremos bibliotecas como Chart.js para visualização de dados e relatórios auto-gerados que auxiliam na tomada de decisões estratégicas.

---

**Definição 8: Seção de Feedback do Cliente**  
**Componente:** Formulário de Feedback  
**Explicação do componente:** Um formulário simples e intuitivo será disponibilizado no frontend para coleta de feedback dos clientes sobre os itens do cardápio. Os dados coletados serão enviados ao backend para análise, possibilitando melhorias contínuas com base nas opiniões dos usuários.

---

**Definição 9: Funcionalidade Offline**  
**Componente:** Armazenamento Local (IndexedDB)  
**Explicação do componente:** Para garantir o funcionamento em situações de conexão instável, implementaremos uma solução de armazenamento local utilizando IndexedDB nas aplicações web e SQLite nas aplicações móveis. Isso permitirá que os usuários acessem informações essenciais do cardápio e façam pedidos mesmo offline.

---

**Definição 10: Configurações e Preferências do Usuário**  
**Componente:** Interface para Preferências do Usuário  
**Explicação do componente:** Uma seção dedicada permitirá aos usuários configurar suas preferências, como filtros para opções vegetarianas e seleção de categorias do cardápio. As escolha dos usuários serão armazenadas no banco de dados, proporcionando uma experiência mais personalizada.

---

### Conclusão

Essa arquitetura é desenhada para atender a todos os requisitos listados, proporcionando uma solução escalável, segura e eficiente para o sistema de cardápio virtual. Cada componente interage de forma coesa, garantindo que as funcionalidades atendam às necessidades dos restaurantes e ofereçam uma experiência excelente aos clientes. Essa definição irá guiar o arquiteto de software ao tomar decisões sobre tecnologias e implementações específicas.

----

## Etapa 3: Wireframes e Prototipagem

In [12]:
prompt = """
**Contexto:**
Haja como a equipe de UX/UI Design
que desenvolverá os protótipos e wireframes do sistema

Ação:
Você deverá criar um protótipo de média fidelidade, utilizando html, css e javascript (no mesmo arquivo) de modo
a simular a tela de Cadastro de Alimentos do cardápio virtual. Considere que um administrador esteja devidamente
logado e tenha permissão para tal. 

Resultado:
O protótipo deverá conter todos os campos necessários para o cadastro dos alimentos na plataforma

"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


**Contexto:**
Haja como a equipe de UX/UI Design
que desenvolverá os protótipos e wireframes do sistema

Ação:
Você deverá criar um protótipo de média fidelidade, utilizando html, css e javascript (no mesmo arquivo) de modo
a simular a tela de Cadastro de Alimentos do cardápio virtual. Considere que um administrador esteja devidamente
logado e tenha permissão para tal. 

Resultado:
O protótipo deverá conter todos os campos necessários para o cadastro dos alimentos na plataforma



----

*Resposta:*

----

Abaixo está um exemplo de um protótipo de média fidelidade para a tela de Cadastro de Alimentos do cardápio virtual, utilizando HTML, CSS e JavaScript no mesmo arquivo. Este protótipo simula o formulário que um administrador utilizaria para cadastrar um novo item no cardápio.

```html
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Cadastro de Alimentos - Cardápio Virtual</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 20px;
        }
        .container {
            max-width: 600px;
            background: #fff;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
            margin: auto;
        }
        h1 {
            text-align: center;
            color: #333;
        }
        label {
            display: block;
            margin: 10px 0 5px;
        }
        input[type="text"], input[type="number"], textarea, select {
            width: 100%;
            padding: 10px;
            margin: 10px 0;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        input[type="file"] {
            margin: 10px 0;
        }
        button {
            background: #28a745;
            color: white;
            border: none;
            padding: 10px;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
        }
        button:hover {
            background: #218838;
        }
        .alert {
            display: none;
            margin-top: 20px;
            padding: 10px;
            background: #d4edda;
            color: #155724;
            border: 1px solid #c3e6cb;
            border-radius: 5px;
        }
    </style>
</head>
<body>

<div class="container">
    <h1>Cadastro de Alimentos</h1>

    <form id="foodForm">
        <label for="name">Nome do Alimento:</label>
        <input type="text" id="name" name="name" required>

        <label for="description">Descrição:</label>
        <textarea id="description" name="description" rows="4" required></textarea>

        <label for="price">Preço:</label>
        <input type="number" id="price" name="price" step="0.01" required>

        <label for=" ingredients">Ingredientes:</label>
        <textarea id="ingredients" name="ingredients" rows="4" required></textarea>

        <label for="image">Imagem do Alimento:</label>
        <input type="file" id="image" name="image" accept="image/*" required>

        <label for="category">Categoria:</label>
        <select id="category" name="category" required>
            <option value="">Selecione uma categoria</option>
            <option value="entrada">Entrada</option>
            <option value="prato">Prato Principal</option>
            <option value="sobremesa">Sobremesa</option>
            <option value="bebida">Bebida</option>
        </select>

        <label for="allergens">Alergênicos (opcional):</label>
        <input type="text" id="allergens" name="allergens">

        <button type="submit">Cadastrar Alimento</button>
    </form>

    <div class="alert" id="alertMessage">Alimento cadastrado com sucesso!</div>
</div>

<script>
    document.getElementById('foodForm').addEventListener('submit', function (event) {
        event.preventDefault(); // Evita o envio real do formulário para demonstração

        // Aqui você pode implementar a lógica para enviar os dados para o servidor

        // Simula a mensagem de sucesso
        document.getElementById('alertMessage').style.display = 'block';
        setTimeout(() => {
            document.getElementById('alertMessage').style.display = 'none';
        }, 3000); // Mensagem desaparece após 3 segundos
    });
</script>

</body>
</html>
```

### Descrição do Protótipo

1. **Campos do Formulário**:
   - **Nome do Alimento**: Campo para o nome do alimento.
   - **Descrição**: Um campo de texto onde o administrador pode descrever o alimento.
   - **Preço**: Campo numérico para o preço do alimento.
   - **Ingredientes**: Campo para listar os ingredientes do prato.
   - **Imagem do Alimento**: Opção de upload de uma imagem.
   - **Categoria**: Um menu suspenso para selecionar a categoria do alimento (entrada, prato principal, sobremesa, bebida).
   - **Alergênicos**: Campo opcional para listar alérgenos.

2. **Botão de Cadastro**: Um botão que, ao ser pressionado, simula o envio do formulário e exibe uma mensagem de sucesso.

3. **Mensagem de Sucesso**: Após o envio do formulário, uma mensagem de sucesso é exibida por 3 segundos para simular a confirmação de que o alimento foi cadastrado.

### Como Testar
- Para testar, copie o código em um arquivo HTML e abra-o em um navegador web. Você verá a interface de cadastro e poderá interagir com os campos e simular a adição de alimentos.

----

## Etapa 8 - Testes de Funcionalidade e Usabilidade

In [13]:
prompt = """
**Contexto:**
Haja como a equipe de QA ou UX/UI Design
que será responsável por construir um plano de testes para a tela de cadastro
de alimentos. 

Ação:
Crie um plano de testes no formato 'Azure Test Plan'. Lembre-se dos requisitos dessa
tela e do wireframe que você fez.

Resultado:
Um arquivo csv que seja compatível com o formato 'Azure Test Plan'

Exemplo (das colunas que devem existir neste plano de testes):
"ID", "Work Item Type", "Title", "Test Step", "Step Action", "Step Expected", "Area Path", "Assigned To", "State"
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


**Contexto:**
Haja como a equipe de QA ou UX/UI Design
que será responsável por construir um plano de testes para a tela de cadastro
de alimentos. 

Ação:
Crie um plano de testes no formato 'Azure Test Plan'. Lembre-se dos requisitos dessa
tela e do wireframe que você fez.

Resultado:
Um arquivo csv que seja compatível com o formato 'Azure Test Plan'

Exemplo (das colunas que devem existir neste plano de testes):
"ID", "Work Item Type", "Title", "Test Step", "Step Action", "Step Expected", "Area Path", "Assigned To", "State"


----

*Resposta:*

----

Abaixo está um exemplo de plano de testes no formato CSV compatível com o 'Azure Test Plan', focando na tela de Cadastro de Alimentos. Esse arquivo inclui os requisitos identificados e as interações que podem ser realizadas na tela. Você pode copiar o conteúdo abaixo e salvá-lo em um arquivo chamado `Plano_de_Testes_Cadastro_Alimentos.csv`.

```csv
"ID","Work Item Type","Title","Test Step","Step Action","Step Expected","Area Path","Assigned To","State"
"1","Test Case","Cadastrar Alimento - Verificar Nome","1","Preencher o campo 'Nome do Alimento' com um nome válido.","O nome do alimento deve ser aceito sem erros. ","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"2","Test Case","Cadastrar Alimento - Verificar Descrição","2","Preencher o campo 'Descrição' com uma descrição válida.","A descrição deve ser aceita sem erros.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"3","Test Case","Cadastrar Alimento - Verificar Preço","3","Preencher o campo 'Preço' com um valor numérico positivo.","O preço deve ser aceito sem erros.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"4","Test Case","Cadastrar Alimento - Verificar Ingredientes","4","Preencher o campo 'Ingredientes' com conteúdo válido.","Os ingredientes devem ser aceitos sem erros.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"5","Test Case","Cadastrar Alimento - Verificar Upload de Imagem","5","Selecionar uma imagem válida para o campo 'Imagem do Alimento'.","A imagem deve ser carregada sem erros e deve aparecer na visualização (se aplicável).","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"6","Test Case","Cadastrar Alimento - Verificar Categoria","6","Selecionar uma categoria no menu suspenso.","A categoria deve ser selecionada corretamente sem erros.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"7","Test Case","Cadastrar Alimento - Verificar Alergênicos","7","Preencher o campo 'Alergênicos' com conteúdo opcional.","O campo de alergênicos deve ser aceito sem erros, mesmo que esteja vazio.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"8","Test Case","Cadastrar Alimento - Verificar Botão de Cadastro","8","Clicar no botão 'Cadastrar Alimento'.","Uma mensagem de sucesso deve ser exibida e o formulário deve ser limpo.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"9","Test Case","Cadastrar Alimento - Verificar Campos Obrigatórios","9","Tentar enviar o formulário com campos obrigatórios vazios.","Deve ser exibida uma mensagem de erro indicando que todos os campos obrigatórios precisam ser preenchidos.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
"10","Test Case","Cadastrar Alimento - Verificar Limitações de Tamanho","10","Inserir texto muito longo em 'Nome do Alimento' ou 'Descrição'.","Deve haver uma mensagem de erro ou a entrada deve ser truncada se exceder o limite definido.","Cardápio Virtual - Cadastro de Alimentos","Nome do Testador","New"
```

### Descrição do Conteúdo

- **ID**: Identificador único de cada caso de teste.
- **Work Item Type**: Tipo de item de trabalho, neste caso, todos são "Test Case".
- **Title**: Título descritivo do caso de teste.
- **Test Step**: Identificador da etapa do teste.
- **Step Action**: Ação a ser realizada durante o teste.
- **Step Expected**: Resultado esperado após a execução da ação.
- **Area Path**: Caminho da área ao qual o teste está associado, ajudando na organização.
- **Assigned To**: Nome do testador responsável pelo caso de teste (pode ser preenchido com o nome real posteriormente).
- **State**: Estado do caso de teste, que pode ser "New", "In Progress" ou "Completed".

### Como Usar

1. Copie o conteúdo acima.
2. Crie um novo arquivo no seu computador e cole o conteúdo.
3. Salve o arquivo com a extensão `.csv`, por exemplo, `Plano_de_Testes_Cadastro_Alimentos.csv`.
4. Importe o arquivo para o Azure DevOps ou use-o como referência para a criação de testes manuais.

O formato CSV permite fácil importação e gerenciamento no Azure Test Plans, facilitando a execução e a verificação dos testes relacionados à tela de Cadastro de Alimentos.

----